In [5]:
import numpy as np

## L-layer Neural Network

In [25]:
class NeuralNetwork():
    def __init__(self, layers):
        """
        layers: python array (list) containing the dimensions of each layer in our network
        for example: layers = [2, 3, 4, 1] is a three-layer NN which input layer has two features
        """
        self.layers = layers
        self.L = len(layers) - 1 # the number of neural network layers should exclude input layer
        
    def initialize_parameters(self):
        """
        Initial parameters (W, b) and store each of them in a parameters dictionary.
        for example: parameters["W1"], parameters["b1"], parameters["W2"], parameters["b2"]
        W[l] is the shape of (n[l], n[l-1]), b[l] is the shape of (n[l], 1)
        
        Arguments:
            - epsilon: a constant to weights
        Return:
            - parameters: a python dict which contains the weights and biases
        """
        #np.random.seed(1)
        parameters = {}
        for l in range(1, self.L+1):
            # implement "He" initialization to weights
            parameters["W" + str(l)] = np.random.randn(self.layers[l], self.layers[l-1]) * np.sqrt(2 / self.layers[l-1])
            parameters["b" + str(l)] = np.zeros((self.layers[l], 1))
            
            assert(parameters["W" + str(l)].shape == (self.layers[l], self.layers[l-1]))
            assert(parameters["b" + str(l)].shape == (self.layers[l], 1))
        
        assert(len(parameters) // 2 == self.L)
            
        return parameters
    
    def parameters_to_vector(self, parameters):
        """
        Roll all parameters dictionary into a single vector
        """
        keys = []
        count = 0
        for key in sorted(parameters.keys()):
            # flatten parameters
            new_vector = np.reshape(parameters[key], (-1, 1))
            keys += [key] * new_vector.shape[0]
            
            if count == 0:
                theta = new_vector
            else:
                theta = np.concatenate((theta, new_vector), axis=0)
            count += 1
        return theta, sorted(keys)
    
    def vector_to_parameters(self, theta, keys):
        """
        Convert the parameters vector back to dictionary
        """
        parameters = {}
        for i, key in enumerate(keys):
            parameters.setdefault(key, []).append(list(theta[i]))
        for l in range(1, self.L + 1):
            parameters["W" + str(l)] = np.array(parameters["W" + str(l)]).reshape(self.layers[l], self.layers[l-1])
            parameters["b" + str(l)] = np.array(parameters["b" + str(l)]).reshape(self.layers[l], 1)
            
        return parameters
    
    def gradient_to_vector(self, grads):
        """
        Convert the gradient dictionary dW/db to a single vector
        """
        keys = list(grads.keys())
        dW_db_keys = []
        # remove dZ and dA keys, only keep dW and db keys
        import re
        for key in keys:
            if re.search(r"d(W|b)", key, re.I) is not None:
                dW_db_keys.append(key)
        count = 0
        for key in sorted(dW_db_keys):
            # flatten dW db gradient
            new_vector = np.reshape(grads[key], (-1, 1))
            if count == 0:
                theta = new_vector
            else:
                theta = np.concatenate((theta, new_vector), axis=0)
            count += 1
        return theta, sorted(dW_db_keys)
    
    def sigmoid(self, Z):
        """
        Sigmoid activation function for the output layer
        """
        Activations = 1. / (1 + np.exp(-Z))
        return Activations
    
    def relu(self, Z):
        """
        RELU activation function for the hidden layer
        """
        Activations = np.maximum(0, Z)
        return Activations
    
    def forward_propagation(self, X, parameters):
        """
        Implement the forward propagation for the entire neural network
        Arguments:
            - X: numpy array of shape (number of features, number of examples)
            - parameters: a python dict which contain the weights and biases
        Return:
            - Z: a list of linear forward function for each layer,start from Z[1] (Z[0] is None)
            - A: a list of activations of Z, A[0] = X
        """
        A = []
        Z = []
        A.append(X) # make A[0] = X
        Z.append(None) # make Z[0] = None (not used)
        for l in range(1, self.L + 1):
            Z.append(np.dot(parameters["W" + str(l)], A[l-1]) + parameters["b"+ str(l)])
            if l == self.L:
                A.append(self.sigmoid(Z[l]))
            else:
                A.append(self.relu(Z[l]))
        
        assert(len(Z) == self.L + 1), print("The dimension of Z is wrong")
        assert(len(A) == self.L + 1), print("The dimension of A is wrong")
        return A, Z
    
    def forward_propagation_with_dropout(self, X, parameters, keep_prob=0.5):
        """
        Implements the forward propagation with dropout
    
        Arguments:
        X -- input dataset, of shape (number of features, number of examples)
        parameters -- python dictionary containing your parameters "W1", "b1", "W2", "b2",..., "WL", "bL":
        keep_prob - probability of keeping a neuron active during drop-out, scalar
    
        Returns:
        A -- a list of linear forward function for each layer,start from Z[1] (Z[0] is None)
        Z -- a list of activations of Z, A[0] = X
        D -- a list of dropout matrix, D[0] = None
        """
        np.random.seed(1)
        A = []
        Z = []
        D = []
        A.append(X) # make A[0] = X
        Z.append(None) # make Z[0] = None (not used)
        D.append(None)
        for l in range(1, self.L + 1):
            Z.append(np.dot(parameters["W" + str(l)], A[l-1]) + parameters["b"+ str(l)])
            if l == self.L:
                A.append(self.sigmoid(Z[l]))
            else:
                A.append(self.relu(Z[l]))
            # Apply dropout to hidden unit
            if l is not self.L:
                Dl = np.random.rand(A[l].shape[0], A[l].shape[1])  # Step 1: initialize matrix Dl = np.random.rand(..., ...)
                Dl = (Dl < keep_prob)                              # Step 2: convert entries of Dl to 0 or 1 (using keep_prob as the threshold)
                A[l] = np.multiply(A[l], Dl)                       # Step 3: shut down some neurons of Al
                A[l] = A[l] / keep_prob                            # Step 4: scale the value of neurons that haven't been shut down
                D.append(Dl)
        return A, Z, D             
    
    def compute_cost(self, Y, AL):
        """
        Implement the cost function
        Argument:
            - Y: the true "label" output eache sample, shape (1, number of examples)
            - AL: the predict output, shape (1, number of examples)
        """
        m = Y.shape[1]
        cost = (-1 / m) * np.sum(Y * np.log(AL) + (1 - Y) * np.log(1-AL))
        cost= np.squeeze(cost) # To make sure your cost's shape is what we expect (e.g. this turns [[17]] into 17)
        assert(cost.shape == ())
        return cost
    
    def compute_cost_with_regularization(self, Y, AL, parameters, lambd):
        """
        Implement the cost function with L2 regularization
        Argument:
            - Y: the true "label" output eache sample, shape (1, number of examples)
            - AL: the predict output, shape (1, number of examples)
            - parameters: python dictionary containing parameters of the model
            - lambd: regularization hyperparameter, scalar
        """
        m = Y.shape[1]
        l2_regularization_cost = 0
        for l in range(1, self.L+1):
            l2_regularization_cost += (lambd / (2 * m)) * np.sum(np.square(parameters["W" + str(l)]))
        cost = self.compute_cost(Y, AL) + l2_regularization_cost
        return cost
    
    def sigmoid_backward(self, dA, Z):
        """
        Implements the backward propagation for SIGMOID unit. 
        """
        s = 1. / (1 + np.exp(-Z))
        dZ = dA * s * (1 - s)
        assert(dZ.shape == Z.shape)
        return dZ
    
    def relu_backward(self, dA, Z):
        """
        Implements the backward propagation for RELU unit
        """
        dZ = np.array(dA, copy=True)
        # When z <= 0, you should set dz to 0 as well. 
        dZ[Z <= 0] = 0
        assert(dZ.shape == Z.shape)
        return dZ
    
    def backward_propagation(self, Y, A, Z, parameters):
        """
        Implement the backward propagation for each layer
        Arguments:
            - Y: The true outputs of each example
            - A: The list of activation function of each layer
            - Z: The list linear forward function of each layer
            - parameters: the weights and biases of each layer
        Return:
            - grads: A dictionary with the gradients
                grads["dA" + str(l)] = ...
                grads["dW" + str(l)] = ...
                grads["db" + str(l)] = ...
        """
        grads = {}
        m = Y.shape[1]
        # Initialize the back propagation
        AL = A[self.L]
        # Way 1
        grads["dA" + str(self.L)] = -(np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
        grads["dZ" + str(self.L)] = self.sigmoid_backward(grads["dA" + str(self.L)], Z[self.L])
        grads["dW" + str(self.L)] = (1. / m) * np.dot(grads["dZ" + str(self.L)], A[self.L-1].T)
        grads["db" + str(self.L)] = (1. / m) * np.sum(grads["dZ" + str(self.L)], axis=1, keepdims=True)
        for l in reversed(range(self.L - 1)):
            grads["dA" + str(l + 1)] = np.dot(parameters["W" + str(l + 2)].T, grads["dZ" + str(l + 2)])
            grads["dZ" + str(l + 1)] = self.relu_backward(grads["dA" + str(l + 1)], Z[l + 1])
            grads["dW" + str(l + 1)] = (1. / m) * np.dot(grads["dZ" + str(l + 1)], A[l].T)
            grads["db" + str(l + 1)] = (1. / m) * np.sum(grads["dZ" + str(l + 1)], axis=1, keepdims=True)
              
        return grads
    
    def backward_propagation_with_regularization(self, Y, A, Z, parameters, lambd=0.5):
        """
        Implement the backward propagation for each layer
        Arguments:
            - Y: The true outputs of each example
            - A: The list of activation function of each layer
            - Z: The list linear forward function of each layer
            - parameters: the weights and biases of each layer
            - lambd: regularization hyperparameter, scalar
        Return:
            - grads: A dictionary with the gradients
                grads["dA" + str(l)] = ...
                grads["dW" + str(l)] = ...
                grads["db" + str(l)] = ...
        """
        grads = {}
        m = Y.shape[1]
        # Initialize the back propagation
        AL = A[self.L]
        # Way 1
        grads["dA" + str(self.L)] = -(np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
        grads["dZ" + str(self.L)] = self.sigmoid_backward(grads["dA" + str(self.L)], Z[self.L])
        grads["dW" + str(self.L)] = (1. / m) * np.dot(grads["dZ" + str(self.L)], A[self.L-1].T) + (lambd / m) * parameters["W" + str(self.L)]
        grads["db" + str(self.L)] = (1. / m) * np.sum(grads["dZ" + str(self.L)], axis=1, keepdims=True)
        for l in reversed(range(self.L - 1)):
            grads["dA" + str(l + 1)] = np.dot(parameters["W" + str(l + 2)].T, grads["dZ" + str(l + 2)])
            grads["dZ" + str(l + 1)] = self.relu_backward(grads["dA" + str(l + 1)], Z[l + 1])
            grads["dW" + str(l + 1)] = (1. / m) * np.dot(grads["dZ" + str(l + 1)], A[l].T) + (lambd / m) * parameters["W" + str(l + 1)]
            grads["db" + str(l + 1)] = (1. / m) * np.sum(grads["dZ" + str(l + 1)], axis=1, keepdims=True)
              
        return grads
    
    def backward_propagation_with_dropout(self, Y, A, Z, D, parameters, keep_prob=1):
        """
        Implement the backward propagation with dropout for each layer
        Arguments:
            - Y: The true outputs of each example
            - A: The list of activation function of each layer
            - Z: The list linear forward function of each layer
            - parameters: the weights and biases of each layer
            - lambd: regularization hyperparameter, scalar
        Return:
            - grads: A dictionary with the gradients
                grads["dA" + str(l)] = ...
                grads["dW" + str(l)] = ...
                grads["db" + str(l)] = ...
        """
        grads = {}
        m = Y.shape[1]
        # Initialize the back propagation
        AL = A[self.L]
        # Way 1
        grads["dA" + str(self.L)] = -(np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
        grads["dZ" + str(self.L)] = self.sigmoid_backward(grads["dA" + str(self.L)], Z[self.L])
        grads["dW" + str(self.L)] = (1. / m) * np.dot(grads["dZ" + str(self.L)], A[self.L-1].T)
        grads["db" + str(self.L)] = (1. / m) * np.sum(grads["dZ" + str(self.L)], axis=1, keepdims=True)
        for l in reversed(range(self.L - 1)):
            grads["dA" + str(l + 1)] = np.dot(parameters["W" + str(l + 2)].T, grads["dZ" + str(l + 2)])
            grads["dA" + str(l + 1)] = np.multiply(grads["dA" + str(l + 1)], D[l + 1]) / keep_prob
            grads["dZ" + str(l + 1)] = self.relu_backward(grads["dA" + str(l + 1)], Z[l + 1])
            grads["dW" + str(l + 1)] = (1. / m) * np.dot(grads["dZ" + str(l + 1)], A[l].T)
            grads["db" + str(l + 1)] = (1. / m) * np.sum(grads["dZ" + str(l + 1)], axis=1, keepdims=True)
              
        return grads
    
    def update_parameters(self, parameters, grads, learning_rate):
        """
        Update parameters using gradient descent
    
        Arguments:
        parameters -- python dictionary containing your parameters 
        grads -- python dictionary containing your gradients, output of L_model_backward
    
        Returns:
        parameters -- python dictionary containing your updated parameters 
                      parameters["W" + str(l)] = ... 
                      parameters["b" + str(l)] = ...
        """
        for l in range(self.L):
            parameters["W" + str(l+1)] -= learning_rate * grads["dW" + str(l + 1)]
            parameters["b" + str(l+1)] -= learning_rate * grads["db" + str(l + 1)]
            
        return parameters
    
    def training_(self, X, Y, n_iters=1000, learning_rate=0.01, print_cost=False):
        """
        Training this Neural Network
        """
        #np.random.seed(1)
        costs = []
        # Initialize parameters
        parameters = self.initialize_parameters()
        
        for i in range(n_iters):
            # forward propagation
            A, Z = self.forward_propagation(X, parameters)
        
            # compute cost
            cost = self.compute_cost(Y, A[self.L])
            
            # backward propagation
            grads = self.backward_propagation(Y, A, Z, parameters)
            
            # update parameters
            parameters = self.update_parameters(parameters, grads, learning_rate)
        
            if print_cost and i % 100 == 0:
                print(f"Cost at iteration {i}: {cost}")
            
            if print_cost and i % 100 == 0:
                costs.append(cost)
        
        #pprint.pprint(grads)
        # plot the cost
        plt.plot(np.squeeze(costs))
        plt.ylabel("cost")
        plt.xlabel("Number of iteration")
        plt.title("Learning rate = " + str(learning_rate))
        plt.show()
                
        return parameters
    
    def training(self, X, Y, n_iters=1000, learning_rate=0.01, print_cost=False, lambd=0, keep_prob=1, gradient_check=False):
        """
        Training this Neural Network
        Argument:
            X - input data, of shape (input size, number of examples)
            Y - true "label" vector (1 for blue dot / 0 for red dot), of shape (output size, number of examples)
            learning_rate - learning rate of the optimization
            num_iterations - number of iterations of the optimization loop
            print_cost - If True, print the cost every 100 iterations
            lambd - regularization hyperparameter, scalar
            keep_prob - probability of keeping a neuron active during drop-out, scalar
            
        Returns:
            parameters - parameters learned by the model. They can then be used to predict.
        """
        #np.random.seed(1)
        costs = []
        m = X.shape[1]
        # Initialize parameters
        parameters = self.initialize_parameters()
        
        for i in range(n_iters):
            # forward propagation
            if keep_prob == 1:
                A, Z = self.forward_propagation(X, parameters)
            elif keep_prob < 1:
                A, Z, D = self.forward_propagation_with_dropout(X, parameters, keep_prob)
        
            # compute cost
            if lambd == 0:
                cost = self.compute_cost(Y, A[-1])
            else:
                cost = self.compute_cost_with_regularization(Y, A[-1], parameters, lambd)
            
            # backward propagation
            if lambd == 0 and keep_prob == 1:
                grads = self.backward_propagation(Y, A, Z, parameters)
            elif lambd != 0:
                grads = self.backward_propagation_with_regularization(Y, A, Z, parameters, lambd)
            elif keep_prob < 1:
                grads = self.backward_propagation_with_dropout(Y, A, Z, D, parameters, keep_prob)
            
            # Apply gradient check for several iterations
            if gradient_check and i in range(3):
                self.gradient_check(parameters, grads, X, Y)
                
            # update parameters
            parameters = self.update_parameters(parameters, grads, learning_rate)
        
            if print_cost and i % 10000 == 0:
                print(f"Cost at iteration {i}: {cost}")
            
            if print_cost and i % 1000 == 0:
                costs.append(cost)
        
        #pprint.pprint(grads)
        # plot the cost
        plt.plot(np.squeeze(costs))
        plt.ylabel("cost")
        plt.xlabel("Number of iteration")
        plt.title("Learning rate = " + str(learning_rate))
        plt.show()
                
        return parameters
    
    def gradient_check(self, parameters, gradients, X, Y, epsilon=1e-7):
        """
        Checks if backward_propagation computes correctly the gradient of the cost output by forward_propagation
    
        Arguments:
            parameters -- python dictionary containing your parameters
            gradients -- output of backward_propagation, contains gradients of the cost with respect to the parameters. 
            X -- input data, of shape (input size, number of examples)
            Y - true "label" vector, of shape (output size, number of examples)
            epsilon -- tiny shift to the input to compute approximated gradient
    
        Returns:
            difference -- difference between the approximated gradient and the backward propagation gradient
        """
        # dict to vector
        parameters_values, keys = self.parameters_to_vector(parameters)
        grads_values, _ = self.gradient_to_vector(gradients)
        num_parameters = parameters_values.shape[0]
        J_plus = np.zeros((num_parameters, 1))
        J_minus = np.zeros((num_parameters, 1))
        gradapprox = np.zeros((num_parameters, 1))
        
        # Compute gradapprox
        for i in range(num_parameters):
            # Compute J_plus[i]
            thetaplus = np.copy(parameters_values)
            thetaplus[i][0] += epsilon
            Aplus, _ = self.forward_propagation(X, self.vector_to_parameters(thetaplus, keys))
            J_plus[i] = self.compute_cost(Y, Aplus[-1])
            
            # Compute J_minus[i]
            thetaminus = np.copy(parameters_values)
            thetaminus[i][0] -= epsilon
            Aminus, _ = self.forward_propagation(X, self.vector_to_parameters(thetaminus, keys))
            J_minus[i] = self.compute_cost(Y, Aminus[-1])
            
            gradapprox[i] = (J_plus[i] - J_minus[i]) / (2 * epsilon)
            
        numerator = np.linalg.norm(gradapprox - grads_values)
        denominator = np.linalg.norm(gradapprox) + np.linalg.norm(grads_values)
        difference = numerator / denominator
        
        if difference > 1e-7:
            print("There is a mistake in backward propagation! difference = " + str(difference))
        else:
            print ("Your backward propagation works perfectly fine! difference = " + str(difference))
            
        return difference
    
    def predict(self, X, Y, parameters):
        """
        Using the learned parameters, predicts a class for each example in X
    
        Arguments:
        parameters -- python dictionary containing your parameters 
        X -- input data of size (n_x, m)
        Y -- the true label of examples
    
        Returns
        predictions -- vector of predictions of our model (red: 0 / blue: 1)
        """
        m = Y.shape[1]
        A, Z = self.forward_propagation(X, parameters)
        Y_predict = A[self.L]
        predictions = np.zeros((1, m))
        #predictions = np.array([1 if elem > 0.5 else 0 for elem in Y_predict[0, :]])
        for i in range(0, Y_predict.shape[1]):
            if Y_predict[0, i] > 0.5:
                predictions[0, i] = 1
            else:
                predictions[0, i] = 0
        print("Accuracy: "  + str(np.sum((predictions == Y)/m)))
        return predictions